In [2]:
from qutip import *
import numpy as np
from scipy import integrate
from helper_code_qutip import * 
import scipy.io

In [2]:
NL1 = 1
NL2 = 1
NM = 2

N = NL1 + NL2 + NM
dL1 = 2**NL1
dL2 = 2**NL2
dM = 2**NM
d = 2**N
dims = [2]*N

create_sm_list_left = [create_sm(N,i + 1) for i in range(NL1)]
create_sm_list_right = [create_sm(N,NM + NL1 + i + 1) for i in range(NL2)]

In [4]:
## Firstly, we have to define the integral function C and D

def integral1(i,k,tb,beta,mu,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(func1,0,b,args=(tb,beta,mu,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(func2,0,b,args=(tb,beta,mu,gamma),limit=limit_value)[0]
    return integral

def integral2(i,k,tb,gamma,eigenergies,limit_value = 700,b=50):
    freq=eigenergies[k]-eigenergies[i]
    if( np.absolute(freq) >= 1/10**10):
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b,args=(tb,gamma),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]
    else:
        integral = (-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b,args=(tb,gamma),limit=limit_value)[0]
    return integral

def C(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + 0.5*(func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))

    return val

def D(i,k,tb,beta,mu,gamma,eigenergies):
    val = integral1(i,k,tb,beta,mu,gamma,eigenergies) + integral2(i,k,tb,gamma,eigenergies) + 0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma)+func1(eigenergies[k]-eigenergies[i],tb,beta,mu,gamma))
    return val

def L2_red(rho,eigstates,number, constant11,constant12,constant21,constant22):
    sum = 0
    rho = Qobj(rho)
    rho.dims = [dims,dims]
    for i in range(number):
        for k in range(number):
            vi = eigstates[i]
            vk = eigstates[k]

            proj_i = vi*vi.dag()
            proj_k = vk*vk.dag()

            for l in range(NL1):
                op = commutator(rho*proj_i*create_sm_list_left[l]*proj_k,create_sm_list_left[l].dag())*constant11[i,k] + commutator(create_sm_list_left[l].dag(),proj_i*create_sm_list_left[l]*proj_k*rho)*constant12[i,k]
                sum += op
                sum += op.dag()

            for l in range(NL2):
                op = commutator(rho*proj_i*create_sm_list_right[l]*proj_k,create_sm_list_right[l].dag())*constant21[i,k] + commutator(create_sm_list_right[l].dag(),proj_i*create_sm_list_right[l]*proj_k*rho)*constant22[i,k]
                sum += op
                sum += op.dag()
    data = sum.full()
    sum = np.array(data,dtype=complex)
    return sum

def re_ness_beta(beta_r,g,ham_type,e):
    #Define the parameters
    print("Beta_r is ",beta_r)
    w0list = np.linspace(1,1,N)
    for i in range(int(N/2),N):
        w0list[i] = 1 + e
    glist = np.linspace(g,g,N-1)
    delta = 1
    beta1 = beta_r  #right baths
    beta2_list = [1.0,1.2,1.5,1.7,2.0,2.2,2.4,2.7,3.0,3.5]#np.logspace(-1,1,12)
    #Define the Hamiltonian
    if ham_type == 1:
        H_S = create_hamiltonian(w0list,glist,delta,N)
    elif ham_type == 2:
        H_S = create_hamiltonian2(w0list,glist,N)
    eigenergies,eigstates=H_S.eigenstates()
    number = len(eigenergies)
    #Define the change of basis unitary matrix
    U = np.zeros((number,number),dtype=complex)
    for i in range(number):
        U[:,i] = eigstates[i].full().flatten()

    gamma1 = 1
    gamma2 = 1
    limit_value = 700
    b_val = 500
    mu1 = 0
    mu2 = 0
    epsilon = 0.01
    tb = 0.01

    rho_ness_arr = []
    l2_red_arr = []

    for index in range(len(beta2_list)):
        
        
        beta2 = beta2_list[index]
        #print("Beta2 is ",beta2)

        #rho_th = (-beta2*H_S).expm()/((-beta2*H_S).expm()).tr() #left thermal density matrix
        #print(rho_th)
        number = len(eigenergies)
        integral11=np.empty((number,number),dtype=np.cdouble) #stores J * N integral for left bath
        integral12=np.empty((number,number),dtype=np.cdouble) # stores J integral (just to check) for the left bath
        integral21=np.empty((number,number),dtype=np.cdouble) #stores J*N integral for right bath
        integral22=np.empty((number,number),dtype=np.cdouble)

                #print("Integral calculations at beta2 = {} and e = {} are : ".format(beta2,e))

        for i in range(number):
            for k in range(number):
                        freq=eigenergies[k]-eigenergies[i]
                        #print(f"Absolute frequency  for i = {i}, k = {k} is ",np.absolute(freq))
                        #print(f"Absolute frequency  for i = {i}, k = {k} is ",np.absolute(freq))
                        #print(i,k,freq)
                        if( np.absolute(freq) >= 1/10**10):
                            integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b_val,args=(tb,beta2,mu2,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                            integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b_val,args=(tb,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #left bath done
                            integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b_val,args=(tb,beta1,mu1,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                            integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b_val,args=(tb,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #right bath
                
                        if (np.absolute(freq)<=1/10**10):
                            #The problem is arising here....
                            integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b_val,args=(tb,beta2,mu2,gamma1),limit=limit_value)[0]
                            integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b_val,args=(tb,gamma1),limit=limit_value)[0]
                            integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b_val,args=(tb,beta1,mu1,gamma2),limit=limit_value)[0]
                            integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b_val,args=(tb,gamma2),limit=limit_value)[0]
                        
                    
                    #expected=1.0j*(eigenergies[k]-eigenergies[i])/(2*tb*tb)
                #        print(i,k,integral2[i,k],expected)
            
            
                # PAY ATTENTION TO THE WAY THESE COEFFICIENTS ARE BEING COMPUTED
            
        constant12=np.empty((number,number),dtype=np.cdouble)
        constant11=np.empty((number,number),dtype=np.cdouble)
        constant21=np.empty((number,number),dtype=np.cdouble)
        constant22=np.empty((number,number),dtype=np.cdouble)
                
                
                
        for i in range(number):
                for k in range(number):
                        constant12[i,k]=integral12[i,k]+integral11[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma1)+func1(eigenergies[k]-eigenergies[i],tb,beta2,mu2,gamma1))    #full coefficient created this is nbar+1
                        constant11[i,k]=integral11[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta2,mu2,gamma1)                                       # the full coefficient is created
                        
                        constant22[i,k]=integral22[i,k]+integral21[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma2)+func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2))    #full coefficient created this is nbar+1
                        constant21[i,k]=integral21[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2)   # the full coefficient is created
                        #print(i,k,constant11[i,k],constant12[i,k],constant21[i,k],constant22[i,k])

    

        ## Now we will write out the matrix elements

        A = np.zeros((number,number),dtype=complex)

        for i in range(number):
            for k in range(number):
                sum = 0
                vi = eigstates[i]
                vk = eigstates[k]
                proj_i = vi*vi.dag()
                proj_k = vk*vk.dag()
                for y in range(number):
                    #print("i = ",i,"k = ",k,"y = ",y)
                    for l in range(NL1):
                        proj_y = eigstates[y]*eigstates[y].dag()
                        op1 = commutator(proj_k*create_sm_list_left[l]*proj_y,create_sm_list_left[l].dag())*constant11[k,y]
                        sum += epsilon*epsilon*vi.dag()*(op1 + op1.dag())*vi

                        op2 = commutator(create_sm_list_left[l].dag(),proj_y*create_sm_list_left[l]*proj_k)*constant12[y,k]
                        sum += epsilon*epsilon*vi.dag()*(op2 + op2.dag())*vi

                    for l in range(NL2):
                        proj_y = eigstates[y]*eigstates[y].dag()
                        op1 = commutator(proj_k*create_sm_list_right[l]*proj_y,create_sm_list_right[l].dag())*constant21[k,y]
                        sum += epsilon*epsilon*vi.dag()*(op1 + op1.dag())*vi

                        op2 = commutator(create_sm_list_right[l].dag(),proj_y*create_sm_list_right[l]*proj_k)*constant22[y,k]
                        sum += epsilon*epsilon*vi.dag()*(op2 + op2.dag())*vi

                A[i,k] = sum

        b = np.zeros((number),dtype=complex)
        A_new = A[:-1]
        A_new = np.vstack([A_new,np.ones((1,number))])
        b[-1] = 1  ## Last element of b is 1

        x = np.linalg.solve(A_new,b)

        #print("Correctness check:",np.dot(A_new,x))
        #print(np.dot(A[-1],x))

        x_real = [np.real(x[i]) for i in range(number)]

        rho = np.diag(x_real)

        #set U matrix whose columns are the eigenvectors of the Hamiltonian

        

        rho_comp2 = np.dot(U,np.dot(rho,U.T.conjugate()))
        print(rho_comp2)
        print()
        rho_ness_arr.append(rho_comp2)
        L2_redfield = L2_red(rho_comp2,eigstates,number,constant11,constant12,constant21,constant22)
        l2_red_arr.append(L2_redfield)

    data_dict = {"dm_ness":rho_ness_arr,"L2_red":l2_red_arr, "beta2":beta2, "g":g, "e":e, "ham_type":ham_type}

    scipy.io.savemat(f'../matlab/data_plotting_vsbeta_4/ness_data_NL1={NL1},NL2={NL2},NM={NM},e={round(e,2)},beta_r={beta_r},g={round(g,4)}_{ham_type}.mat',data_dict)

def re_ness_g(beta_r,beta_l,ham_type,e):
    #Define the parameters
    print("Beta_r is ",beta_r, "and Beta_l is ",beta_l)
    w0list = np.linspace(1,1,N)
    for i in range(int(N/2),N):
        w0list[i] = 1 + e
   
    delta = 1
    beta1 = beta_r  #right baths
    beta2 = beta_l

    gvals = np.logspace(-2,1,12)
    

    gamma1 = 1
    gamma2 = 1
    limit_value = 700
    b_val = 50
    mu1 = -1e-10
    mu2 = -1e-10
    epsilon = 0.01
    tb = 0.01

    rho_ness_arr = []
    l2_red_arr = []

    for index in range(len(gvals)):
        
        
        g = gvals[index]
        glist = np.linspace(g,g,N-1)
        #Define the Hamiltonian
        if ham_type == 1:
            H_S = create_hamiltonian3(w0list,glist,delta,N)
        elif ham_type == 2:
            H_S = create_hamiltonian2(w0list,glist,N)
        eigenergies,eigstates=H_S.eigenstates()
        number = len(eigenergies)
        #Define the change of basis unitary matrix
        U = np.zeros((number,number),dtype=complex)
        for i in range(number):
            U[:,i] = eigstates[i].full().flatten()


        #rho_th = (-beta2*H_S).expm()/((-beta2*H_S).expm()).tr() #left thermal density matrix
        #print(rho_th)
        number = len(eigenergies)
        integral11=np.empty((number,number),dtype=np.cdouble) #stores J * N integral for left bath
        integral12=np.empty((number,number),dtype=np.cdouble) # stores J integral (just to check) for the left bath
        integral21=np.empty((number,number),dtype=np.cdouble) #stores J*N integral for right bath
        integral22=np.empty((number,number),dtype=np.cdouble)

                #print("Integral calculations at beta2 = {} and e = {} are : ".format(beta2,e))

        for i in range(number):
            for k in range(number):
                        freq=eigenergies[k]-eigenergies[i]
                        #print(f"Absolute frequency  for i = {i}, k = {k} is ",np.absolute(freq))
                        #print(i,k,freq)
                        if( np.absolute(freq) >= 1/10**10):
                            integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b_val,args=(tb,beta2,mu2,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                            integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b_val,args=(tb,gamma1),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #left bath done
                            integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func1,0,b_val,args=(tb,beta1,mu1,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0] #func 1
                            integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath,0,b_val,args=(tb,gamma2),limit=limit_value,weight='cauchy',wvar=eigenergies[k]-eigenergies[i])[0]  #right bath
                
                        if (np.absolute(freq)<=1/10**10):  #The problem is arising here....
                            integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b_val,args=(tb,beta2,mu2,gamma1),limit=limit_value)[0]
                            integral12[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b_val,args=(tb,gamma1),limit=limit_value)[0]
                            integral21[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b_val,args=(tb,beta1,mu1,gamma2),limit=limit_value)[0]
                            integral22[i,k]=(-1.0j/(2*np.pi))*integrate.quad(spectral_bath_2,0,b_val,args=(tb,gamma2),limit=limit_value)[0]
                        
                    
                    #expected=1.0j*(eigenergies[k]-eigenergies[i])/(2*tb*tb)
                #        print(i,k,integral2[i,k],expected)
            
            
                # PAY ATTENTION TO THE WAY THESE COEFFICIENTS ARE BEING COMPUTED
            
        constant12=np.empty((number,number),dtype=np.cdouble)
        constant11=np.empty((number,number),dtype=np.cdouble)
        constant21=np.empty((number,number),dtype=np.cdouble)
        constant22=np.empty((number,number),dtype=np.cdouble)
                
                
                
        for i in range(number):
                for k in range(number):
                        constant12[i,k]=integral12[i,k]+integral11[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma1)+func1(eigenergies[k]-eigenergies[i],tb,beta2,mu2,gamma1))    #full coefficient created this is nbar+1
                        constant11[i,k]=integral11[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta2,mu2,gamma1)                                       # the full coefficient is created
                        
                        constant22[i,k]=integral22[i,k]+integral21[i,k]+0.5*(spectral_bath(eigenergies[k]-eigenergies[i],tb,gamma2)+func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2))    #full coefficient created this is nbar+1
                        constant21[i,k]=integral21[i,k]+0.5*func1(eigenergies[k]-eigenergies[i],tb,beta1,mu1,gamma2)   # the full coefficient is created
                        #print(i,k,constant11[i,k],constant12[i,k],constant21[i,k],constant22[i,k])

    

        ## Now we will write out the matrix elements

        A = np.zeros((number,number),dtype=complex)

        for i in range(number):
            for k in range(number):
                sum = 0
                vi = eigstates[i]
                vk = eigstates[k]
                proj_i = vi*vi.dag()
                proj_k = vk*vk.dag()
                for y in range(number):
                    for l in range(NL1):
                        proj_y = eigstates[y]*eigstates[y].dag()
                        op1 = commutator(proj_k*create_sm_list_left[l]*proj_y,create_sm_list_left[l].dag())*constant11[k,y]
                        sum += epsilon*epsilon*vi.dag()*(op1 + op1.dag())*vi

                        op2 = commutator(create_sm_list_left[l].dag(),proj_y*create_sm_list_left[l]*proj_k)*constant12[y,k]
                        sum += epsilon*epsilon*vi.dag()*(op2 + op2.dag())*vi

                    for l in range(NL2):
                        proj_y = eigstates[y]*eigstates[y].dag()
                        op1 = commutator(proj_k*create_sm_list_right[l]*proj_y,create_sm_list_right[l].dag())*constant21[k,y]
                        sum += epsilon*epsilon*vi.dag()*(op1 + op1.dag())*vi

                        op2 = commutator(create_sm_list_right[l].dag(),proj_y*create_sm_list_right[l]*proj_k)*constant22[y,k]
                        sum += epsilon*epsilon*vi.dag()*(op2 + op2.dag())*vi

                A[i,k] = sum

        b = np.zeros((number),dtype=complex)
        A_new = A[:-1]
        A_new = np.vstack([A_new,np.ones((1,number))])
        b[-1] = 1  ## Last element of b is 1

        x = np.linalg.solve(A_new,b)

        """print("Correctness check:",np.dot(A_new,x))
        print(np.dot(A[-1],x))"""

        x_real = [np.real(x[i]) for i in range(number)]

        rho = np.diag(x_real)

        #set U matrix whose columns are the eigenvectors of the Hamiltonian

        

        rho_comp2 = np.dot(U,np.dot(rho,U.T.conjugate()))
        rho_ness_arr.append(rho_comp2)
        L2_redfield = L2_red(rho_comp2,eigstates,number,constant11,constant12,constant21,constant22)
        l2_red_arr.append(L2_redfield)

    data_dict = {"dm_ness":rho_ness_arr,"L2_red":l2_red_arr, "beta2":beta2, "g":g, "e":e, "ham_type":ham_type}

    scipy.io.savemat(f'../matlab/data_plotting_vsg_3/ness_data_NL1={NL1},NL2={NL2},NM={NM},e={e},beta_r={beta_r},beta_l={beta_l}_{ham_type}.mat',data_dict)



        

In [22]:
#re_ness_beta(0.5,0.0016,1,0)
re_ness_beta(1,0.0016,1,0)
re_ness_beta(1.5,0.0016,1,0)
#re_ness_beta(10,0.0016,1,0)

Beta_r is  1


/var/folders/ys/hsq5k3zs70b8y915g7tk_yx00000gn/T/ipykernel_49964/1843621385.py:117: IntegrationWarning: The maximum number of subdivisions (700) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  integral11[i,k]=(-1.0j/(2*np.pi))*integrate.quad(func2,0,b_val,args=(tb,beta2,mu2,gamma1),limit=limit_value)[0]
/var/folders/ys/hsq5k3zs70b8y915g7tk_yx00000gn/T/ipykernel_49964/1843621385.py:119: IntegrationWarning: The maximum number of subdivisions (700) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be d

[[ 5.25128563e-03+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  1.42289418e-02+0.j  4.54597617e-05+0.j
   0.00000000e+00+0.j  7.26967645e-08+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  7.75845002e-11+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  4.54597617e-05+0.j  1.41835547e-02+0.j
   0.00000000e+00+0.j  4.53871426e-05+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  7.26967645e-08+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 

/Users/soumyadeepsarma/Desktop/Projects Soumyadeep/ICTS/SearchingLindbladians/python/helper_code_qutip.py:207: RuntimeWarning: overflow encountered in exp
  return 1/(np.exp(beta*(omega-mu))-1)


[[ 2.93415953e-03+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  9.64907832e-03+0.j  3.64597821e-05+0.j
   0.00000000e+00+0.j  6.84565109e-08+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j -3.43984642e-10+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 [ 0.00000000e+00+0.j  3.64597821e-05+0.j  9.61268700e-03+0.j
   0.00000000e+00+0.j  3.63909816e-05+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  6.84565109e-08+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
   0.00000000e+00+0.j]
 

In [8]:
re_ness_g(0.5,1,1,0.01)
re_ness_g(1,1,1,0.01)
re_ness_g(5,1,1,0.01)
re_ness_g(10,1,1,0.01)

Beta_r is  0.5 and Beta_l is  1
Beta_r is  1 and Beta_l is  1
Beta_r is  5 and Beta_l is  1
Beta_r is  10 and Beta_l is  1


In [9]:
re_ness_g(0.5,1,2,0)
re_ness_g(1,1,2,0)
re_ness_g(5,1,2,0)
re_ness_g(10,1,2,0)

Beta_r is  0.5 and Beta_l is  1
Beta_r is  1 and Beta_l is  1
Beta_r is  5 and Beta_l is  1
Beta_r is  10 and Beta_l is  1


In [10]:
re_ness_g(0.5,1,2,0.01)
re_ness_g(1,1,2,0.01)
re_ness_g(5,1,2,0.01)
re_ness_g(10,1,2,0.01)
             

Beta_r is  0.5 and Beta_l is  1
Beta_r is  1 and Beta_l is  1
Beta_r is  5 and Beta_l is  1
Beta_r is  10 and Beta_l is  1


In [13]:
re_ness_g(0.5,1,2,0)
re_ness_g(1,1,2,0)
re_ness_g(5,1,2,0)
re_ness_g(10,1,2,0)

Beta_r is  0.5 and Beta_l is  1
Beta_r is  1 and Beta_l is  1
Beta_r is  5 and Beta_l is  1
Beta_r is  10 and Beta_l is  1
